In [ ]:
# Let's explore the eigenstate wigner functions too now

To do so, we will vary various parameters to see when the hamiltonian due to RWA is significantly different in comparison to the the (quantum rabi) model including terms that do not preserve total excitation number.

As discussed in the theory and deeivation, we are comparing the following 2 hamiltonians:




Loaded in python Qutip

In [2]:
from qutip import *

# g : coupling strength
# wr : resonator frequency
# wt : qubit frequency
# N : truncates resonator hilbert space

def HamiltonianJaynesCummings(g : float, wr : float, wt : float, N : int) -> Qobj:
    a = destroy(N)
    return tensor(a.dag()*a, qeye(2))*wr + 0.5*wr - 0.5*wt*tensor(qeye(N), sigmaz()) + g*(tensor(a.dag(),sigmap()) + tensor(a,sigmam()))

def HamiltonianRabi(g : float, wr : float, wt : float, N : int) -> Qobj:
    a = destroy(N)
    return tensor(a.dag()*a, qeye(2))*wr + 0.5*wr - 0.5*wt*tensor(qeye(N), sigmaz()) + g*tensor(a.dag() + a, qeye(2))*tensor(qeye(N), sigmap() + sigmam())

In the code, the tensor products are included to match the hamiltonian dimensions. As the live in a combined space of resonator and two level system. Let us create these hamiltonians for a small resonator space and see if the difference in structure. 

In addition, hbar is approximated as 1, as it is only a scalar!

In [3]:
N = 3
wt = 1.0
wr = 1.0
g = 0.1 

In comparison to the Quantum Rabi model

We are explicitly truncating the resonator hilbert space to N.

## Exploring Wigner Functions and density matrix representation

In [ ]:
# Similarity between Wigner functions (ground state , nth eigenstate).

In [ ]:
import numpy as np
from qutip import wigner, ptrace, Qobj
from ipywidgets import interactive, FloatSlider, IntSlider, Layout, VBox, Output
from IPython.display import display
import plotly.graph_objects as go
import matplotlib

N = 10  # Truncation of the resonator Hilbert space
wr = 1.0 
X_WIG = np.linspace(-5, 5, 100) # Smaller grid for static embedding
Y_WIG = np.linspace(-5, 5, 100)

plot_output = Output()

def plot_nth_wigner_plotly(g, det, n):
    # We will plot the Nth eigenstate Wigner function of the Jaynes-Cummings Hamiltonian
    H_JC = HamiltonianJaynesCummings(g, wr, wr + det, N)
    # nth eigenstate of H_JC
    nth_state = H_JC.eigenstates()[1][n]

    nth_state_reduced = nth_state.ptrace(0)
    W_data = wigner(nth_state_reduced, X_WIG, Y_WIG)
    
    cmap = matplotlib.colormaps.get_cmap('RdBu')
    
    fig = go.Figure(data=[go.Heatmap(
        z=W_data,
        x=X_WIG,
        y=Y_WIG,
        colorscale='RdBu', # Plotly's internal RdBu is fine
        zmin=np.min(W_data),
        zmax=np.max(W_data)
    )])
    
    fig.update_layout(
        title=f'Wigner Function of Eigenstate n={n} (g={g}, Δ={det})',
        xaxis_title="Re(α)",
        yaxis_title="Im(α)",
        width=600,
        height=600,
        yaxis=dict(scaleanchor="x", scaleratio=1), # Force 1:1 aspect ratio
    )
    
    # Crucially, we update the Output widget contents
    with plot_output:
        plot_output.clear_output(wait=True)
        fig.show()

g_slider = FloatSlider(
    min=-3.0, max=3.0, step=0.1, value=0.0,
    description="Coupling Strength (g)",
    layout=Layout(width='90%'),
    style={'description_width': 'initial'}
)

detuning_slider = FloatSlider(
    min=-3.0, max=3.0, step=0.1, value=0.0,
    description="Detuning (Δ)",
    layout=Layout(width='90%'),
    style={'description_width': 'initial'}
)

n_slider = IntSlider(
    min=0, max=5, step=1, value=0,
    description="Eigenstate Index (n)",
    layout=Layout(width='90%'),
    style={'description_width': 'initial'}
)

interactive_widgets = interactive(
    plot_nth_wigner_plotly, 
    g=g_slider,
    det=detuning_slider, 
    n=n_slider
)

display(VBox([
    g_slider, 
    detuning_slider, 
    n_slider, 
    plot_output 
]))

plot_nth_wigner_plotly(g_slider.value, detuning_slider.value, n_slider.value)